In [ ]:
! pip install -U langchain_community langchain-openai langchain langgraph bs4

  Using cached langchain_community-0.2.12-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain_openai-0.1.22-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain-0.2.14-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 500.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s e

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Tutorial"

In [ ]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)
concatenated_content

'\n\n\n\n\nConceptual guide | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain\n\n\n\n\n\n\n\nSkip to main contentLangChain 0.2 is out! Leave feedback on the v0.2 docs here. You can view the v0.1 docs here.IntegrationsAPI referenceLatestLegacyMorePeopleContributingCookbooks3rd party tutorialsYouTubearXivv0.2v0.2v0.1\uf8ffü¶úÔ∏è\uf8ffüîóLangSmithLangSmith DocsLangChain HubJS/TS Docs\uf8ffüí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selector

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

### OpenAI

# Grader prompt
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a coding assistant with expertise in LCEL, LangChain expression language. \n
    Here is a full set of LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user
    question based on the above provided documentation. Ensure any code you provide can be executed \n
    with all required imports and variables defined. Structure your answer with a description of the code solution. \n
    Then list the imports. And finally list the functioning code block. Please ensure to use the latest OpenAI model, gpt-4o-mini.
  Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)

# Data model
class code(BaseModel):
    """Code output"""
    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")
    description = "Schema for code solutions to questions about LCEL."

expt_llm = "gpt-4o-mini"
llm = ChatOpenAI(temperature=0, model=expt_llm)
code_gen_chain = code_gen_prompt | llm.with_structured_output(code)
# question = "How do I build a RAG chain in LCEL?"
# solution = code_gen_chain.invoke({"context":concatenated_content,"messages":[("user",question)]})

In [ ]:
# Test
question = "How do I build a RAG chain in LCEL?"
solution = code_gen_chain.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)
solution

code(prefix='This code demonstrates how to build a Retrieval-Augmented Generation (RAG) chain using LangChain Expression Language (LCEL). The RAG chain combines a retriever to fetch relevant documents and a language model to generate responses based on those documents.', imports='from langchain_core.prompts import ChatPromptTemplate\nfrom langchain_openai import ChatOpenAI\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.retrievers import VectorStoreRetriever\nfrom langchain_core.vectorstores import FAISS\nfrom langchain_core.embeddings import OpenAIEmbeddings\n\n# Initialize the OpenAI model\nmodel = ChatOpenAI(model="gpt-4o-mini")\n\n# Create a vector store and retriever\nembeddings = OpenAIEmbeddings()\nvector_store = FAISS(embedding_function=embeddings)\nretriever = VectorStoreRetriever(vector_store=vector_store)', code='# Define the prompt template for the RAG chain\nprompt = ChatPromptTemplate.from_template("Given the following documents, answer the

In [ ]:
from typing import List, TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        error : Binary flag for control flow to indicate whether test error was tripped
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
    """

    error: str
    messages: List
    generation: str
    iterations: int

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

### Parameter

# Max tries
max_iterations = 3
# Reflect
# flag = 'reflect'
flag = "do not reflect"

### Nodes
def generate(state: GraphState):
    """
    Generate a code solution
    Args:
        state (dict): The current graph state
    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]

    # We have been routed back to generation with an error
    if error == "yes":
        messages += [
            (
                "user",
                "Now, try again. Invoke the code tool to structure the output with a prefix, imports, and code block:",
            )
        ]

    # Solution
    code_solution = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [
        (
            "assistant",
            f"{code_solution.prefix} \n Imports: {code_solution.imports} \n Code: {code_solution.code}",
        )
    ]

    # Increment
    iterations = iterations + 1
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


def code_check(state: GraphState):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("---CHECKING CODE---")

    # State
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # Get solution components
    imports = code_solution.imports
    code = code_solution.code

    # Check imports
    try:
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the import test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # Check execution
    try:
        exec(imports + "\n" + code)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the code execution test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # No errors
    print("---NO CODE TEST FAILURES---")
    return {
        "generation": code_solution,
        "messages": messages,
        "iterations": iterations,
        "error": "no",
    }


def reflect(state: GraphState):
    """
    Reflect on errors

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    code_solution = state["generation"]

    # Prompt reflection

    # Add reflection
    reflections = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [("assistant", f"Here are reflections on the error: {reflections}")]
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


### Edges
def decide_to_finish(state: GraphState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        if flag == "reflect":
            return "reflect"
        else:
            return "generate"

In [ ]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation solution
workflow.add_node("check_code", code_check)  # check code
workflow.add_node("reflect", reflect)  # reflect

# Build graph
workflow.add_edge(START, "generate")
workflow.add_edge("generate", "check_code")
workflow.add_conditional_edges(
    "check_code",
    decide_to_finish,
    {
        "end": END,
        "reflect": "reflect",
        "generate": "generate",
    },
)
workflow.add_edge("reflect", "generate")
app = workflow.compile()

In [ ]:
question = "How can I directly pass a string to a runnable and use it to construct the input needed for my prompt?"
app.invoke({"messages": [("user", question)], "iterations": 0})

---GENERATING CODE SOLUTION---
---CHECKING CODE---
content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 14, 'total_tokens': 34}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None} id='run-89120007-af3c-481b-8835-23324858ce9d-0' usage_metadata={'input_tokens': 14, 'output_tokens': 20, 'total_tokens': 34}
---NO CODE TEST FAILURES---
---DECISION: FINISH---


{'error': 'no',
 'messages': [('user',
   'How can I directly pass a string to a runnable and use it to construct the input needed for my prompt?'),
  ('assistant',
   'To directly pass a string to a runnable and use it to construct the input for a prompt in LangChain, you can utilize the `Runnable` interface. This allows you to create a prompt template that takes a string input and formats it accordingly. Below is an example demonstrating how to achieve this using the latest OpenAI model, `gpt-4o-mini`. \n Imports: from langchain_core.prompts import PromptTemplate\nfrom langchain_openai import ChatOpenAI\n\n# Define the prompt template\nprompt_template = PromptTemplate.from_template("""Tell me a joke about {topic}.""")\n\n# Initialize the chat model\nmodel = ChatOpenAI(model="gpt-4o-mini")\n\n# Create a runnable chain that combines the prompt and model\nchain = prompt_template | model \n Code: # Define the topic string\ninput_string = "cats"\n\n# Invoke the chain with the input string

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define the prompt template
prompt_template = PromptTemplate.from_template("""Tell me a joke about {topic}.""")
# Initialize the chat model
model = ChatOpenAI(model="gpt-4o-mini")
# Create a runnable chain that combines the prompt and model
chain = prompt_template | model
# Define the topic string
input_string = "cats"
# Invoke the chain with the input string
response = chain.invoke({'topic': input_string})
# Print the response
print(response)

content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 14, 'total_tokens': 34}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None} id='run-140c9d74-f321-4f19-adf3-1833d89a7520-0' usage_metadata={'input_tokens': 14, 'output_tokens': 20, 'total_tokens': 34}


# 测试集

In [ ]:
import langsmith
client = langsmith.Client()

In [ ]:
# Clone the dataset to your tenant to use it
public_dataset = (
    "https://smith.langchain.com/public/326674a6-62bd-462d-88ae-eea49d503f9d/d"
)
client.clone_public_dataset(public_dataset)

In [ ]:
from langsmith.schemas import Example, Run


def check_import(run: Run, example: Example) -> dict:
    imports = run.outputs.get("imports")
    try:
        exec(imports)
        return {"key": "import_check", "score": 1}
    except Exception:
        return {"key": "import_check", "score": 0}


def check_execution(run: Run, example: Example) -> dict:
    imports = run.outputs.get("imports")
    code = run.outputs.get("code")
    try:
        exec(imports + "\n" + code)
        return {"key": "code_execution_check", "score": 1}
    except Exception:
        return {"key": "code_execution_check", "score": 0}

In [ ]:
def predict_base_case(example: dict):
    """Context stuffing"""
    solution = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": [("user", example["question"])]}
    )
    solution_structured = code_gen_chain.invoke([("code", solution)])
    return {"imports": solution_structured.imports, "code": solution_structured.code}


def predict_langgraph(example: dict):
    """LangGraph"""
    graph = app.invoke({"messages": [("user", example["question"])], "iterations": 0})
    solution = graph["generation"]
    return {"imports": solution.imports, "code": solution.code}

In [ ]:
from langsmith.evaluation import evaluate

# Evaluator
code_evalulator = [check_import, check_execution]

# Dataset
dataset_name = "lcel-teacher-eval"

In [ ]:
# Run base case
experiment_results_ = evaluate(
    predict_base_case,
    data=dataset_name,
    evaluators=code_evalulator,
    experiment_prefix=f"test-without-langgraph-{expt_llm}",
    max_concurrency=2,
    metadata={
        "llm": expt_llm,
    },
)

View the evaluation results for experiment: 'test-without-langgraph-gpt-4o-mini-8d53e367' at:
https://smith.langchain.com/o/97965422-56ca-5e42-94ee-b511c4b3c617/datasets/ad69a8e2-ee48-454d-aee6-c4b4a437e1b7/compare?selectedSessions=5fe0a6ea-21cc-4434-9b27-c63d19297a21




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: Error code: 428 - {'error': {'message': '已触发低积分多请求限制(15RPM),请充值或者过1分钟后重试', 'type': 'openai_hk_error'}}
ERROR:langsmith.evaluation._runner:Error running target function: Error code: 428 - {'error': {'message': '已触发低积分多请求限制(15RPM),请充值或者过1分钟后重试', 'type': 'openai_hk_error'}}
ERROR:langsmith.evaluation._runner:Error running target function: Error code: 428 - {'error': {'message': '已触发低积分多请求限制(15RPM),请充值或者过1分钟后重试', 'type': 'openai_hk_error'}}
ERROR:langsmith.evaluation._runner:Error running target function: Error code: 428 - {'error': {'message': '已触发低积分多请求限制(15RPM),请充值或者过1分钟后重试', 'type': 'openai_hk_error'}}
ERROR:langsmith.evaluation._runner:Error running target function: Error code: 428 - {'error': {'message': '已触发低积分多请求限制(15RPM),请充值或者过1分钟后重试', 'type': 'openai_hk_error'}}
ERROR:langsmith.evaluation._runner:Error running target function: Error code: 428 - {'error': {'message': '已触发低积分多请求限制(15RPM),请充值或者过1分钟后重试', 'type': 'opena

<class 'pydantic.v1.main.PromptInput'>
Please answer the question: 'where did harrison work' in italian.


In [ ]:
# Run with langgraph
experiment_results = evaluate(
    predict_langgraph,
    data=dataset_name,
    evaluators=code_evalulator,
    experiment_prefix=f"test-with-langgraph-{expt_llm}-{flag}",
    max_concurrency=2,
    metadata={
        "llm": expt_llm,
        "feedback": flag,
    },
)

View the evaluation results for experiment: 'test-with-langgraph-gpt-4o-mini-do not reflect-ea442c35' at:
https://smith.langchain.com/o/97965422-56ca-5e42-94ee-b511c4b3c617/datasets/ad69a8e2-ee48-454d-aee6-c4b4a437e1b7/compare?selectedSessions=b708b5f0-6fd5-4a3b-9746-9aac9f943257




0it [00:00, ?it/s]

---GENERATING CODE SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!' response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ba606877f9', 'finish_reason': 'stop', 'logprobs': None} id='run-56aabff9-c8c4-4b0b-8867-76b3012bc08f-0' usage_metadata={'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33}
---NO CODE TEST FAILURES---
---DECISION: FINISH---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---


ERROR:langsmith.evaluation._runner:Error running target function: 1 validation error for code
prefix
  field required (type=value_error.missing)


---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---DECISION: FINISH---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: FINISH---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: FIN

...

ERROR:langsmith.evaluation._runner:Error running target function: Error code: 428 - {'error': {'message': '已触发低积分多请求限制(15RPM),请充值或者过1分钟后重试', 'type': 'openai_hk_error'}}
ERROR:langsmith.evaluation._runner:Error running target function: Error code: 428 - {'error': {'message': '已触发低积分多请求限制(15RPM),请充值或者过1分钟后重试', 'type': 'openai_hk_error'}}


---GENERATING CODE SOLUTION---
---GENERATING CODE SOLUTION---


ERROR:langsmith.evaluation._runner:Error running target function: Error code: 428 - {'error': {'message': '已触发低积分多请求限制(15RPM),请充值或者过1分钟后重试', 'type': 'openai_hk_error'}}


---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---


ERROR:langsmith.evaluation._runner:Error running target function: Error code: 428 - {'error': {'message': '已触发低积分多请求限制(15RPM),请充值或者过1分钟后重试', 'type': 'openai_hk_error'}}
